In [1]:
# constants
DATASET = "quora"
sbert_model_name = "msmarco-distilbert-base-tas-b"
device = "cpu" # cuda for gpu usage
k_queries = 5000
k_documents = 261465

In [2]:
# import libraries
from time import time
from beir import util
from beir_reengineered import NewSentenceBERT
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES
import os, json, random
import pickle
import json

/u/tsouth/.local/lib/python3.10/site-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
2024-04-26 20:21:52.177676: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-26 20:21:52.178511: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-26 20:21:52.204896: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-26 20:21:55.929191: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized 

In [3]:
# Part 2. (only sometimes) round embeddings
from beir.retrieval.search import BaseSearch # type: ignore beir/retrieval/search/dense/exact_search.py
from beir.util import cos_sim
import torch # type: ignore
import numpy as np
from typing import Dict
import heapq
import logging
logger = logging.getLogger(__name__)

In [4]:
# ExperiementRetrievalExactSearch is parent class for any model we are using for our experiement that can be used for retrieval
# Abstract class is BaseSearch
class ExperiementRetrievalExactSearch(BaseSearch):
    def __init__(
            self,
            model,
            path_corpus_embeddings: str = "datasets/subquora/corpus_embeddings.pkl",
            path_query_embeddings: str = "datasets/subquora/query_embeddings.pkl",
            **kwargs):
        #model is class should do nothing
        self.model = model
        self.path_corpus_embeddings = path_corpus_embeddings
        self.path_query_embeddings = path_query_embeddings
        self.show_progress_bar = kwargs.get("show_progress_bar", True)
        self.convert_to_tensor = kwargs.get("convert_to_tensor", True)
        self.results = {}

        logger.info("Load in Encoded Queries and Corpus from Pickle...")
        # Verify file existence
        if not os.path.exists(self.path_corpus_embeddings):
            raise FileNotFoundError(f"File '{self.path_corpus_embeddings}' not found.")
        if not os.path.exists(self.path_query_embeddings):
            raise FileNotFoundError(f"File '{self.path_query_embeddings}' not found.")

        with open(self.path_query_embeddings, "rb") as f:
            self.query_embeddings = pickle.load(f)

        with open(self.path_corpus_embeddings, "rb") as f:
            self.corpus_embeddings = pickle.load(f)



    def add_rounding(self, rounding_decimal: int) -> None:
        # rounding decimal
        if rounding_decimal < 12:
            logger.info("Rounding decimal places of Queries and Corpus...")
            for key, value in self.query_embeddings.items():
                self.query_embeddings[key] = np.round(value, decimals=rounding_decimal)

            for key, value in self.corpus_embeddings.items():
                self.corpus_embeddings[key] = np.round(value, decimals=rounding_decimal)

    def add_noise(self, rounding_decimal: int) -> None:
        logger.info("Adding Noise to Queries and Corpus...")
        for key, value in self.query_embeddings.items():
            self.query_embeddings[key] += np.random.random() / 10**rounding_decimal

        for key, value in self.corpus_embeddings.items():
            self.corpus_embeddings[key] += np.random.random() / 10**rounding_decimal


    def search(self,
               corpus: Dict[str, Dict[str, str]],
               queries: Dict[str, str],
               top_k: int,
               score_function: str,
               return_sorted: bool = False,
               **kwargs) -> Dict[str, Dict[str, float]]:
        # Runs semantic search against the corpus embeddings
        # Returns a ranked list with the corpus ids

        query_ids = list(self.query_embeddings.keys())
        self.results = {qid: {} for qid in query_ids}

        # print("Sorting Corpus by document length (Longest first)...")
        logger.info("Sorting Corpus by document length (Longest first)...")
        corpus_ids = sorted(list(self.corpus_embeddings.keys()), reverse=True)

        result_heaps = {qid: [] for qid in query_ids}  # Keep only the top-k docs for each query

        # Convert dictionary values to PyTorch tensors
        corpus_tensors = [torch.tensor(embedding) for embedding in self.corpus_embeddings.values()]
        query_tensors = [torch.tensor(embedding) for embedding in self.query_embeddings.values()]
        # Stack tensors along a new dimension (batch dimension)
        corpus_embeddings_tensor = torch.stack(corpus_tensors)
        query_embeddings_tensor = torch.stack(query_tensors)

        # print("Compute similarites using  cosine-similarity")
        # Compute similarites using  cosine-similarity
        cos_scores = cos_sim(query_embeddings_tensor, corpus_embeddings_tensor)
        # print("cos_sim after")
        cos_scores[torch.isnan(cos_scores)] = -1

        # print("Get top-k values")
        # Get top-k values
        cos_scores_top_k_values, cos_scores_top_k_idx = torch.topk(cos_scores, min(top_k+1, len(cos_scores[1])), dim=1, largest=True, sorted=return_sorted)
        cos_scores_top_k_values = cos_scores_top_k_values.cpu().tolist()
        cos_scores_top_k_idx = cos_scores_top_k_idx.cpu().tolist()

        # print("build heap")
        for query_itr in range(len(query_embeddings_tensor)):
            query_id = query_ids[query_itr]
            for sub_corpus_id, score in zip(cos_scores_top_k_idx[query_itr], cos_scores_top_k_values[query_itr]):
                corpus_id = corpus_ids[sub_corpus_id]
                if corpus_id != query_id:
                    if len(result_heaps[query_id]) < top_k:
                        # Push item on the heap
                        heapq.heappush(result_heaps[query_id], (score, corpus_id))
                    else:
                        # If item is larger than the smallest in the heap, push it on the heap then pop the smallest element
                        heapq.heappushpop(result_heaps[query_id], (score, corpus_id))

        print("get results heaps")
        for qid in result_heaps:
            for score, corpus_id in result_heaps[qid]:
                self.results[qid][corpus_id] = score

        return self.results

In [5]:
beir_sbert = NewSentenceBERT(sbert_model_name, device=device)

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

In [6]:
model = ExperiementRetrievalExactSearch(
    beir_sbert,
    path_corpus_embeddings = "datasets/subquora/corpus_embeddings_half2.pkl",
    path_query_embeddings= "datasets/subquora/query_embeddings_half2.pkl") # This can go outside the loop

print("here")
with open("datasets/subquora/qrels_half2.json", "r") as f:
    qrels = json.load(f)

with open("datasets/subquora/query_embeddings_half2.pkl", "rb") as f:
    query_embeddings = pickle.load(f)

with open("datasets/subquora/corpus_embeddings_half2.pkl", "rb") as f:
    corpus_embeddings = pickle.load(f)


rounding_recalls = []
rounding_decimals = range(12,-1,-1)
for decimal in rounding_decimals:
    print(decimal, type(decimal))
    model.add_rounding(decimal)
    print("after add_rounding")
    retriever = EvaluateRetrieval(model, score_function="cos_sim")
    print("after EvaluateRetrieval")
    # print(model.corpus_embeddings, model.query_embeddings)
    # issue maybe you have to pass the correct corpus and queries in
    results = retriever.retrieve(corpus_embeddings, query_embeddings)
    print("after retrieve")
    ndcg, _map, recall, precision = retriever.evaluate(qrels, results, retriever.k_values)
    print("decimal_places:", decimal , ndcg, _map, recall, precision )
    #recall@100 matters mostr, save to array for plotting
    rounding_recalls.append(recall)


here
12 <class 'int'>
after add_rounding
after EvaluateRetrieval
get results heaps
after retrieve
decimal_places: 12 {'NDCG@1': 0.0, 'NDCG@3': 0.0, 'NDCG@5': 0.0, 'NDCG@10': 0.0, 'NDCG@100': 5e-05, 'NDCG@1000': 0.00051} {'MAP@1': 0.0, 'MAP@3': 0.0, 'MAP@5': 0.0, 'MAP@10': 0.0, 'MAP@100': 1e-05, 'MAP@1000': 2e-05} {'Recall@1': 0.0, 'Recall@3': 0.0, 'Recall@5': 0.0, 'Recall@10': 0.0, 'Recall@100': 0.00021, 'Recall@1000': 0.00402} {'P@1': 0.0, 'P@3': 0.0, 'P@5': 0.0, 'P@10': 0.0, 'P@100': 0.0, 'P@1000': 1e-05}
11 <class 'int'>
after add_rounding
after EvaluateRetrieval
get results heaps
after retrieve
decimal_places: 11 {'NDCG@1': 0.0, 'NDCG@3': 0.0, 'NDCG@5': 0.0, 'NDCG@10': 0.0, 'NDCG@100': 5e-05, 'NDCG@1000': 0.00051} {'MAP@1': 0.0, 'MAP@3': 0.0, 'MAP@5': 0.0, 'MAP@10': 0.0, 'MAP@100': 1e-05, 'MAP@1000': 2e-05} {'Recall@1': 0.0, 'Recall@3': 0.0, 'Recall@5': 0.0, 'Recall@10': 0.0, 'Recall@100': 0.00021, 'Recall@1000': 0.00402} {'P@1': 0.0, 'P@3': 0.0, 'P@5': 0.0, 'P@10': 0.0, 'P@100': 0

In [7]:
noise_recalls = []
noise_decimals = range(12,-1,-1)
for decimal in range(12,-1,-1):
    # print(decimal, type(decimal))
    model.add_noise(decimal)
    # model.noise()
    retriever = EvaluateRetrieval(model, score_function="cos_sim")
    results = retriever.retrieve(model.corpus_embeddings, model.query_embeddings)
    ndcg, _map, recall, precision = retriever.evaluate(qrels, results, retriever.k_values)
    print("noise decimal:", decimal , ndcg, _map, recall, precision )
    noise_recalls.append(recall)

get results heaps
noise decimal: 12 {'NDCG@1': 0.0, 'NDCG@3': 0.0, 'NDCG@5': 0.0, 'NDCG@10': 0.0, 'NDCG@100': 1e-05, 'NDCG@1000': 0.0005} {'MAP@1': 0.0, 'MAP@3': 0.0, 'MAP@5': 0.0, 'MAP@10': 0.0, 'MAP@100': 0.0, 'MAP@1000': 1e-05} {'Recall@1': 0.0, 'Recall@3': 0.0, 'Recall@5': 0.0, 'Recall@10': 0.0, 'Recall@100': 4e-05, 'Recall@1000': 0.00364} {'P@1': 0.0, 'P@3': 0.0, 'P@5': 0.0, 'P@10': 0.0, 'P@100': 0.0, 'P@1000': 1e-05}
get results heaps
noise decimal: 11 {'NDCG@1': 0.0, 'NDCG@3': 0.0, 'NDCG@5': 0.0, 'NDCG@10': 0.0, 'NDCG@100': 1e-05, 'NDCG@1000': 0.0005} {'MAP@1': 0.0, 'MAP@3': 0.0, 'MAP@5': 0.0, 'MAP@10': 0.0, 'MAP@100': 0.0, 'MAP@1000': 1e-05} {'Recall@1': 0.0, 'Recall@3': 0.0, 'Recall@5': 0.0, 'Recall@10': 0.0, 'Recall@100': 4e-05, 'Recall@1000': 0.00364} {'P@1': 0.0, 'P@3': 0.0, 'P@5': 0.0, 'P@10': 0.0, 'P@100': 0.0, 'P@1000': 1e-05}
get results heaps
noise decimal: 10 {'NDCG@1': 0.0, 'NDCG@3': 0.0, 'NDCG@5': 0.0, 'NDCG@10': 0.0, 'NDCG@100': 1e-05, 'NDCG@1000': 0.0005} {'MAP@1'

In [8]:
import csv
# Headers for CSV file
headers = ['Rounding_Decimals', 'Recall@100', 'Recall@1000']

# Create and write data to CSV file
with open('results/rounding_recalls_half.csv', 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=headers)
    writer.writeheader()

    for rounding_decimal, rounding_recall in zip(rounding_decimals, rounding_recalls):
        row_data = {
            'Rounding_Decimals': rounding_decimal,
            'Recall@100': rounding_recall['Recall@100'],
            'Recall@1000': rounding_recall['Recall@1000']
        }
        writer.writerow(row_data)

In [9]:
# Headers for CSV file
headers = ['Rounding_Decimals', 'Recall@100', 'Recall@1000']

# Create and write data to CSV file
with open('results/noise_recall_half.csv', 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=headers)
    writer.writeheader()

    for rounding_decimal, noise_recall in zip(rounding_decimals, noise_recalls):
        row_data = {
            'Rounding_Decimals': rounding_decimal,
            'Recall@100': noise_recall['Recall@100'],
            'Recall@1000': noise_recall['Recall@1000']
        }
        writer.writerow(row_data)